<a href="https://colab.research.google.com/github/junieberry/DL-fromScratch2/blob/main/ToyProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NLP Project : 한국어 영화 리뷰 감정 분석

## 데이터 column 정보

- id : naver에서 제공하는 id
- document : 리뷰
- label : 레이블(0 : negative, 1: positive)

In [1]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [2]:
import pandas as pd

## 필드 구문이 탭으로 되어 있어서 \t로 구분자를 지정해준다.
train = pd.read_csv("nsmc/ratings_train.txt","\t")
test = pd.read_csv("nsmc/ratings_test.txt","\t")

print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


In [3]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 데이터 전처리
Konlpy를 사용한 `형태소 분석 및 품사 태깅


In [ ]:
!pip install konlpy
%env JAVA_HOME "C:\Program Files\Java\jdk-14.0.2"

In [8]:
import konlpy
from konlpy.tag import Okt

okt = Okt()

print("okt.morph >> ",okt.morphs(u"우리는 파이썬에서 정한 규칙을 벗어났기 때문에 에러를 만나게 됩니다."))
print("okt.nouns >> ", okt.nouns(u"파이썬은 에러가 발생한경우에 굉장히 친절하게 알려주는 언어입니다."))
print("okt.phrases >>" , okt.phrases(u'파이썬 에러 발생한 문장을 보면, 에러가 발생한 파일이름과 몇번째 줄인지, 그리고 에러가 발생한 문장도 함께 보여줍니다.'))
print("okt.pos >>", okt.pos(u"에러를 만난다면, 당황하지 않고 먼저 읽으시기 바랍니다."))
print("okt.pos >>", okt.pos(u"ㅋㅋㅋㅋ야 이거 실화냐? 개꿀잼"))
print("okt.pos >>", okt.pos(u"그렇게 하면 안돼"))
print("okt.pos >>", okt.pos(u"그러케 하면 않되"))

okt.morph >>  ['우리', '는', '파이썬', '에서', '정', '한', '규칙', '을', '벗어났기', '때문', '에', '에러', '를', '만나게', '됩니다', '.']
okt.nouns >>  ['파이썬', '에러', '발생', '경우', '언어']
okt.phrases >> ['파이썬', '파이썬 에러', '파이썬 에러 발생', '문장', '에러', '발생', '파일이름', '파일이름과 몇번째', '파일이름과 몇번째 줄', '몇번째']
okt.pos >> [('에러', 'Noun'), ('를', 'Josa'), ('만난다면', 'Verb'), (',', 'Punctuation'), ('당황', 'Noun'), ('하지', 'Verb'), ('않고', 'Verb'), ('먼저', 'Noun'), ('읽으시기', 'Verb'), ('바랍니다', 'Verb'), ('.', 'Punctuation')]
okt.pos >> [('ㅋㅋㅋㅋ', 'KoreanParticle'), ('야', 'Exclamation'), ('이', 'Determiner'), ('거', 'Noun'), ('실화', 'Noun'), ('냐', 'Josa'), ('?', 'Punctuation'), ('개꿀잼', 'Noun')]
okt.pos >> [('그렇게', 'Adverb'), ('하면', 'Verb'), ('안', 'VerbPrefix'), ('돼', 'Verb')]
okt.pos >> [('그러케', 'Noun'), ('하면', 'Verb'), ('않되', 'Verb')]


In [11]:
def tokenize(sentence):
  ## 형태소와 품사를 join
  return ['/'.join(t) for t in okt.pos(sentence, norm=True, stem=True)]

In [12]:
train.isnull().any() #document에 null값이 있다.
train['document'] = train['document'].fillna('')

test.isnull().any()
test['document'] = test['document'].fillna('')

In [13]:
train_docs = [(tokenize(row[1]), row[2]) for row in train.values]
test_docs = [(tokenize(row[1]), row[2]) for row in test.values]

In [14]:
train_docs[2]

(['너/Modifier',
  '무재/Noun',
  '밓었/Noun',
  '다그/Noun',
  '래서/Noun',
  '보다/Verb',
  '추천/Noun',
  '한/Josa',
  '다/Adverb'],
 0)

In [15]:
tokens = [t for d in train_docs for t in d[0]]
print("토큰개수:", len(tokens))

토큰개수: 2159921
